# Using the Edge Impulse Python SDK with TensorFlow and Keras

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://docs.edgeimpulse.com/docs/edge-impulse-python-sdk/01-python-sdk-with-tf-keras"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-ei-32px.png" /> View on edgeimpulse.com</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/edgeimpulse/notebooks/blob/main/notebooks/01-python-sdk-with-tf-keras.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-colab-32px.png" /> Run in Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/edgeimpulse/notebooks/blob/main/notebooks/01-python-sdk-with-tf-keras.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-github-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/notebooks/01-python-sdk-with-tf-keras.ipynb" download><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/icon-download-32px.png" /> Download notebook</a>
  </td>
</table>

[TensorFlow](https://www.tensorflow.org/) is an open source library for training machine learning models. [Keras](https://keras.io/) is an open source Python library that makes creating neural networks in TensorFlow much easier. We use these two libraries together to very quickly train a model to identify handwritten digits. From there, we use the Edge Impulse Python SDK library to profile the model to see how inference will perform on a target edge device. Then, we use the SDK again to convert our trained model to a C++ library that can be deployed to an edge hardware platform, such as a microcontroller.

Follow the code below to see how to train a simple machine learning model and deploy it to a C++ library using Edge Impulse.

To learn more about using the Python SDK, please see: [Edge Impulse Python SDK Overview](https://docs.edgeimpulse.com/docs/edge-impulse-python-sdk/overview).

In [ ]:
# If you have not done so already, install the following dependencies
!python -m pip install tensorflow edgeimpulse

In [1]:
from tensorflow import keras
import edgeimpulse as ei

2023-04-03 16:17:39.144261: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


You will need to obtain an API key from an Edge Impulse project. Log into [edgeimpulse.com](https://edgeimpulse.com/) and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**.

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-copy-ei-api-key.png)

Note that you do not actually need to use the project in the Edge Impulse Studio. We just need the API Key.

Paste that API key string in the `ei.API_KEY` value in the following cell:

In [32]:
# Settings
ei.API_KEY = "ei_dae2..." # Change this to your Edge Impulse API key
labels = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
num_classes = len(labels)
savedmodel_name = "my_model"
deploy_filename = "my_model_cpp.zip"

## Train a machine learning model

We want to create a classifier that can uniquely identify handwritten digits. To start, we will use TensorFlow and Keras to train a very simple convolutional neural network (CNN) on the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, which consists of handwritten digits from 0 to 9.

In [3]:
# Load MNIST data
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = keras.utils.normalize(x_train, axis=1)
x_test = keras.utils.normalize(x_test, axis=1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
input_shape = x_train[0].shape

In [4]:
# Build the model 
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation='relu', input_shape=input_shape),
    keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# Train the model
model.fit(x_train, 
          y_train, 
          epochs=5)

Epoch 1/5
1875/1875 [==============================] - 2s 874us/step - loss: 0.4382 - accuracy: 0.8797
Epoch 2/5
1875/1875 [==============================] - 2s 852us/step - loss: 0.2328 - accuracy: 0.9341
Epoch 3/5
1875/1875 [==============================] - 2s 843us/step - loss: 0.1881 - accuracy: 0.9459
Epoch 4/5
1875/1875 [==============================] - 2s 856us/step - loss: 0.1618 - accuracy: 0.9527
Epoch 5/5
1875/1875 [==============================] - 2s 869us/step - loss: 0.1424 - accuracy: 0.9577


In [6]:
# Evaluate model on test set
score = model.evaluate(x_test, y_test, verbose=0)
print(f"Test loss: {score[0]}")
print(f"Test accuracy: {score[1]}")

Test loss: 0.15037794411182404
Test accuracy: 0.957099974155426


## Profile your model

To start, we need to list the possible target devices we can use for profiling. We need to pick from this list

In [9]:
# List the available profile target devices
ei.model.list_profile_devices()

['alif-he',
 'alif-hp',
 'arduino-nano-33-ble',
 'arduino-nicla-vision',
 'portenta-h7',
 'brainchip-akd1000',
 'cortex-m4f-80mhz',
 'cortex-m7-216mhz',
 'espressif-esp32',
 'himax-we-i',
 'infineon-cy8ckit-062s2',
 'infineon-cy8ckit-062-ble',
 'jetson-nano',
 'mbp-16-2020',
 'nordic-nrf52840-dk',
 'nordic-nrf5340-dk',
 'nordic-nrf9160-dk',
 'raspberry-pi-4',
 'raspberry-pi-rp2040',
 'renesas-ck-ra6m5',
 'renesas-rzv2l-cpu',
 'renesas-rzv2l',
 'st-iot-discovery-kit',
 'wio-terminal',
 'silabs-xg24',
 'silabs-thunderboard-sense-2',
 'sony-spresense',
 'synaptics-ka10000',
 'ti-launchxl',
 'ti-tda4vm']

A common option is the `cortex-m4f-80mhz`, as this is a relatively low-power microcontroller family. From there, we can use the Edge Impulse Python SDK to generate a profile for your model to ensure it fits on your target hardware and meets your timing requirements.

In [10]:
# Estimate the RAM, ROM, and inference time for our model on the target hardware family
try:
    profile = ei.model.profile(model=model,
                               device='cortex-m4f-80mhz')
    print(profile.summary())
except Exception as e:
    print(f"Could not profile: {e}")

INFO:tensorflow:Assets written to: /var/folders/_2/25d5mmg50rv4kk0qsv654_4r0000gn/T/tmp6bh_ecrj/saved_model/assets


INFO:tensorflow:Assets written to: /var/folders/_2/25d5mmg50rv4kk0qsv654_4r0000gn/T/tmp6bh_ecrj/saved_model/assets


Target results for float32:
{   'device': 'cortex-m4f-80mhz',
    'tfliteFileSizeBytes': 103788,
    'isSupportedOnMcu': True,
    'memory': {   'tflite': {'ram': 10172, 'rom': 135184, 'arenaSize': 9932},
                  'eon': {'ram': 7840, 'rom': 112208}},
    'timePerInferenceMs': 2}


Performance on device types:
{   'variant': 'float32',
    'lowEndMcu': {   'description': 'Estimate for a Cortex-M0+ or similar, '
                                    'running at 40MHz',
                     'timePerInferenceMs': 64,
                     'memory': {   'tflite': {'ram': 10121, 'rom': 130752},
                                   'eon': {'ram': 7808, 'rom': 112144}},
                     'supported': True},
    'highEndMcu': {   'description': 'Estimate for a Cortex-M7 or other '
                                     'high-end MCU/DSP, running at 240MHz',
                      'timePerInferenceMs': 2,
                      'memory': {   'tflite': {'ram': 10172, 'rom': 135184},
         

## Deploy your model

Once you are happy with the performance of the model, you can deploy it to a number of possible hardware targets. The most generic target is to download a .zip file containing a C++ library containing the inference runtime and your trained model. To do that, we first need to create a Classification object which contains our label strings (and other optional information about the model). These strings will be added to the C++ library metadata so you can access them in your edge application.

In [ ]:
# Set model information, such as your list of labels
model_output_type = ei.model.output_type.Classification(labels=labels)

# Create C++ library with trained model
deploy_bytes = None
try:
    
    deploy_bytes = ei.model.deploy(model=model,
                                   model_output_type = model_output_type,
                                   deploy_target='zip')
except Exception as e:
    print(f"Could not deploy: {e}")
    
# Write the downloaded raw bytes to a file
if deploy_bytes:
    with open(deploy_filename, 'wb') as f:
        f.write(deploy_bytes)

Your model C++ library should be downloaded as the file *my_model_cpp.zip* in the same directory as this notebook. You are now ready to use your C++ model in your embedded and edge device application! To use the C++ model for local inference, see our documentation [here](https://docs.edgeimpulse.com/docs/deployment/running-your-impulse-locally).